# Dome paradox

The dome, John D. Norton, 2008

https://www.cambridge.org/core/product/identifier/S0031824800011843/type/journal_article

DOI: 10.1086/594524

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sympy import dsolve, symbols, Function, Eq
import sympy
import polars as pl
import torch
import numpy as np
import gif
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from PIL import Image
import math
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController
import jax.numpy as jnp
from jaxtyping import Float, Array
import seaborn as sns
from scipy.integrate import solve_ivp

## Maths

Case description: We have a particle sitting on top of a dome, without velocity or acceleration. A uniform gravitational field may effect the particle.

## Newton

rotationally symmetric dome described in paper with height function

$$ h = \frac{2}{3g} r^{3/2} $$

there is no minus sign here because of the definition of the z-axis, I find this confusing. the hightest point should be at $r=0$. hence

$$ h = - \frac{2}{3g} r^{3/2} $$

so we write out uniform gravitation potential as

$$ V = mgh = - \frac{2m}{3} r^{3/2} $$

$$ F = ma $$

which we just assume to be

$$ F = m\ddot{r} $$

Because 

$$ F = - \frac{dV}{dr} $$

we find

$$ m\ddot{r} = m \sqrt{r} $$

so

$$ \ddot{r} = \sqrt{r} $$

In [ ]:
r = symbols("r", cls=Function)
t = symbols("t")

eq = Eq(r(t).diff(t, t), sympy.sqrt(r(t)))
eq

In [ ]:
res = dsolve(eq)

ugh so we get two solutions

In [ ]:
res[0]

In [ ]:
res[1]

for comparison without the square root we get only one

In [ ]:
eq = Eq(r(t).diff(t, t), r(t))
res = dsolve(eq)

In [ ]:
res

Let's do the Lagrangian / Hamiltonian formalism for the fun of it

### Lagrangian

$$ L = T - V $$

$$ T =  \frac{1}{2} m \left( \dot{q}_x^2 + \dot{q}_y^2 + \dot{q}_z^2 \right)  $$

the uniform  graviational potential is

$$ V_G = m g z $$

shape of the dome

$$ h(r) = - \frac{2}{3g} r^{3/2}$$

where $r$ is the "radial distance coordinate on the surface of the dome". minus sign is a bit weird, but the highest point of the dome needs to be at the origin. 

important is that the dome is radially symmetric. 

since the potential energy should be largest at $ r = 0 $

$$ V_D = m g h(r) $$

$$ x = r \cos \phi $$

$$ y = r \sin \phi $$

$$ \dot{x} = \dot{r} \cos \phi - r \dot{\phi} \sin \phi $$

$$ \dot{y} = \dot{r} \sin \phi + r \dot{\phi} \cos \phi $$

$$ \left( \dot{x}^2 + \dot{y}^2 \right) \rightarrow \left( r^2 \dot\phi^2 + \dot{r}^2 \right) $$



So we have 

$$ T =  \frac{1}{2} m \left( \dot{q}_r^2 \dot{q}_\phi^2 + \dot{q}_r^2 + \dot{q}_z^2 \right)  $$

$$ V_D(q_r) = m g h(q_r) $$

$$ L = T - V = \frac{1}{2} m \left( \dot{q}_r^2 \dot{q}_\phi^2 + \dot{q}_r^2 + \dot{q}_z^2 \right) - m g h(q_r) $$

using 

$$ p = \partial_{\dot{q}} L $$

we get

$$ p_r = \partial_{\dot{q}_r} L = m \dot{q}_r \rightarrow \dot{q}_r = \frac{p_r}{m}$$

$$ p_\phi = \partial_{\dot{q}_\phi} L = 0 $$

$$ p_z = \partial_{\dot{q}_z} L = m \dot{q}_z \rightarrow \dot{q}_z = \frac{p_z}{m}$$

In [ ]:
def get_dome_h(r: Array, g: float) -> Array:
    return -2 / 3.0 / g * r ** (3.0 / 2.0)


r = jnp.linspace(0, 5, 101)
g = 9.81
h = get_dome_h(r, g)

ax = sns.lineplot(x=r, y=h)
ax.set(xlabel="r", ylabel="h", title="Norton's dome h(r)")
plt.tight_layout()

### Hamiltonian

$$ H = \sum_i p_i \dot{q}_i - L $$

$$ H = p_r \dot{q}_r + p_z \dot{q}_z - \frac{1}{2} m \left( \dot{q}_r^2 \dot{q}_\phi^2 + \dot{q}_r^2 + \dot{q}_z^2 \right) + m g h(q_r) $$

$$ H = p_r \frac{p_r}{m} + p_z \frac{p_z}{m} - \frac{1}{2} m \left( \left( \frac{p_r}{m} \right)^2 \dot{q}_\phi^2 + \left( \frac{p_r}{m} \right)^2 + \left( \frac{p_z}{m} \right)^2 \right) + m g h(q_r) $$

$$ H = \frac{1}{2m} \left( p_r^2 + p_z^2 - p_r^2 \dot{q}_\phi^2 \right)  + m g h(q_r) $$

$$ \dot{q}_i = \partial_{p_i} H $$

$$ \dot{p}_i = - \partial_{q_i} H $$

$\phi$:

$$ \dot{q}_\phi = 0 $$

$$ \dot{p}_\phi = 0 $$

$r$:

$$ \dot{q}_r = \frac{p_r}{m} (1 - \dot{q}_\phi^2) \rightarrow \frac{p_r}{m} $$

$$ \dot{p}_r = - mg\partial_{q_r}h(q_r) = -m \sqrt{r} $$

$z$:

$$ \dot{q}_z = \frac{p_z}{m}$$

$$ \dot{p}_z = 0 $$

Since we have $\sqrt{r}$ now as part of $\dot{p}_r$ I guess the analytical solution here also has two solutions, as the Newton version.



Let's solve the equations numerically for fun anyway

initial conditions at $t = 0$:

* $q_r = 0$
* $\dot{q}_r = 0$
* $q_z = 0$
* $\dot{q}_z = 0$

In [ ]:
def ode_to_sole(t, z, args):
    m, g = args
    q_r, q_z, p_r, p_z = z

    q_r_dot = p_r / m
    q_z_dot = p_z / m
    p_r_dot = -m * jnp.sqrt(q_r)
    p_z_dot = 0

    return q_r_dot, q_z_dot, p_r_dot, p_z_dot


m = 1.0
g = 9.81

q_r_0 = 0.0
q_z_0 = 0.0

q_r_dot_0 = 0.0
q_z_dot_0 = 0.0

p_r_0 = q_r_0 * m
p_z_0 = q_z_0 * m


ti, tf = 0, 10
t_span = [ti, tf]
t_eval = jnp.linspace(ti, tf, 101)
term = ODETerm(ode_to_sole)
solver = Dopri5()
saveat = SaveAt(ts=t_eval)
stepsize_controller = PIDController(rtol=1e-5, atol=1e-5)
dt0 = t_eval[1].item() - t_eval[0].item()
params = (m, g)
initial_values = (q_r_0, q_z_0, p_r_0, p_z_0)
sol = diffeqsolve(
    term,
    solver,
    t0=ti,
    t1=tf,
    dt0=dt0,
    y0=initial_values,
    saveat=saveat,
    stepsize_controller=stepsize_controller,
    args=params,
)

In [ ]:
assert sol.ys is not None
t_sol = sol.ts
q_r_sol = sol.ys[0]
q_z_sol = sol.ys[1]
p_r_sol = sol.ys[2]
p_z_sol = sol.ys[3]

fig, axs = plt.subplots(nrows=2, sharex=True)
ax = axs[0]
ax.plot(t_sol, q_r_sol, label="$q_r$")
ax.plot(t_sol, p_r_sol, label="$p_r$")
ax.legend()

ax = axs[1]
ax.plot(t_sol, q_z_sol, label=r"$q_z$")
ax.plot(t_sol, p_z_sol, label=r"$p_z$")
ax.set(xlabel="t", ylabel=r"$z$")
ax.legend()
plt.tight_layout()